Function which uses Poisson distribution to perturb each element in the row count matrix by 5% on average.

In [33]:
import scanpy as sc
import numpy as np

def perturb_dataset(data, p = 0.05):
    n, d = data.shape

    # perturb each row
    for i in range(n):
        row = np.asarray(data[i, :])
        lam = (row * p)
        pois = np.random.poisson(lam, row.shape)
        mask = np.random.random(row.shape) < 0.5 # randomly multiply with -1
        pois[mask] *= -1   

        data[i, :] += pois

    # remove negative values from the matrix    
    data[data < 0] = 0

    return data

Loading, pertrubing and preprocessing the dataset.

In [34]:
def remove_rare_cell_types(adata, labels, threshold):
    n = len(labels)

    # select 'good' labels - labels with at least 50 cells
    cnts = labels.value_counts()
    filtered = set(cnts[cnts >= 50])

    # filter the dataset - first we get the ids
    ids = [i for i in range(n) if labels.iloc[i] in filtered]

    # modify the dataset to remove rare cell types
    if len(ids) < n:
        adata = adata[ids, :].copy()

def get_perturbed_data(p=0.05):
    adata = sc.datasets.paul15()

    adata.X = perturb_dataset(adata.X, p = p)
    labels = adata.obs["paul15_clusters"]

    # preprocess
    remove_rare_cell_types(adata, labels, 50)
    sc.pp.filter_genes(adata, min_cells=10)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    return adata.X, labels

Use SepSolve to select $m$ marker genes on the perturbed dataset.

In [35]:
# import sepsolve
import sys

sys.path.append('../sepsolve')
from sepsolve import get_markers

def get_markers_perturbation(num_markers):
    data, labels = get_perturbed_data(p = 0.05)
    markers = get_markers(data, labels, num_markers)
    return markers

Function which computes the stability index for the given collection of marker genes using Jaccard similarity.

In [36]:
def calculate_stability(marker_collection):
    stability = 0
    inters = set([])
    n = len(marker_collection)
    
    for i in range(n):
        for j in range(i + 1, n):
            inters = marker_collection[i].intersection(marker_collection[j])
            union = marker_collection[i].union(marker_collection[j])
            stability += len(inters) / len(union)
    
    stability *= 2 / (n * (n - 1))

    return stability


Run the perturbation test $k = 5$ times and calculate the perturbation stability index.

In [48]:
k = 5
num_markers = 25

marker_collection = []
for i in range(k):
    print("Running perturbation test", i + 1, flush=True)

    # get markers on the randomly perturbed dataset
    markers = get_markers_perturbation(num_markers)
    marker_collection.append(set(markers))

stability = calculate_stability(marker_collection)

print("Perturbation stability index:", stability)

Running perturbation test 1
Running perturbation test 2
Running perturbation test 3
Running perturbation test 4
Running perturbation test 5
Perturbation stability index: 0.9384615384615386
